In [17]:
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from CurrencyDataset import CurrencyDataset
import math

In [2]:
df = pd.read_csv('../cleaned/usdrub_new.csv')

In [3]:
columns = ['open', 'high', 'low', 'close']
df = df[columns]

In [4]:
training_length = math.floor(len(df) * 0.7)
train_df, test_df = df.iloc[:training_length], df.iloc[training_length:]

In [5]:
train_data = CurrencyDataset(train_df,  seq_length = 5)
test_data = CurrencyDataset(test_df,  seq_length = 5)

In [6]:
train_dl = DataLoader(train_data, batch_size=8)
test_dl = DataLoader(test_data, batch_size=8)

In [22]:
next(iter(train_dl))

{'data': tensor([[[34.8934, 34.9337, 34.5848, 34.7102],
          [34.7300, 34.8300, 34.5300, 34.7500],
          [35.7747, 35.8635, 35.6700, 35.7100],
          [39.7165, 39.9751, 39.6550, 39.8873],
          [39.8959, 40.1560, 39.6877, 39.9721]],
 
         [[34.7300, 34.8300, 34.5300, 34.7500],
          [35.7747, 35.8635, 35.6700, 35.7100],
          [39.7165, 39.9751, 39.6550, 39.8873],
          [39.8959, 40.1560, 39.6877, 39.9721],
          [41.4547, 43.1366, 41.4072, 43.0322]],
 
         [[35.7747, 35.8635, 35.6700, 35.7100],
          [39.7165, 39.9751, 39.6550, 39.8873],
          [39.8959, 40.1560, 39.6877, 39.9721],
          [41.4547, 43.1366, 41.4072, 43.0322],
          [42.9971, 43.0322, 42.9971, 43.0233]],
 
         [[39.7165, 39.9751, 39.6550, 39.8873],
          [39.8959, 40.1560, 39.6877, 39.9721],
          [41.4547, 43.1366, 41.4072, 43.0322],
          [42.9971, 43.0322, 42.9971, 43.0233],
          [43.1758, 43.6175, 43.1471, 43.5873]],
 
         [[39.8959, 

In [19]:
print('Training Set:')
for sample in train_dl:  
    print(sample['data'].size())
    print(sample['target'].size())
    break

Training Set:
torch.Size([8, 5, 4])
torch.Size([8, 4])


In [18]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # out: (batch_size, seq_length, hidden_size)
        out, _ = self.lstm(x)
        # take the out at the last time step
        # out: (batch_size, hidden_size)
        out = out[:, -1]
        # pred: (batch_size, 1)
        pred = self.fc(out)
        # output: batch_size
        return pred.squeeze()